# Создание отчетов из отзывов (часть 1)
В этой серии ноутбуков мы покажем, как из большого количества неструктурированных отзывов пользователей можно выделить тематические группы и проблемные области, а затем на основе полученных данных генерировать отчёты.

_Пример итогового отчёта: [report.pdf](media/report.pdf)_ 

Для анализа используются отзывы с [Kaggle датасета](https://www.kaggle.com/datasets/mikhaildvoshansky/bank-reviews)

**Структура серии:**
1. [Часть 1](part_1.ipynb) – кластеризация и визуальный анализ отзывов.  
2. [Часть 2](part_2.ipynb) – генерация отчёта на основе кластеров с использованием LLM.

**Инструменты:**
- [GigaChat](https://developers.sber.ru/docs/ru/gigachat/api/overview) – генерация эмбеддингов для текста.
- [Arize Phoenix](https://phoenix.arize.com/) – визуализация эмбеддингов и анализ кластеров.

-----
## Методология

Процесс анализа отзывов состоит из следующих этапов:

1. **Предварительная обработка и кластеризация.**  
   Данные, полученные в [части 1](part_1.ipynb), были предварительно обработаны и разбиты на кластеры на основе эмбеддингов.

2. **Выбор ключевых кластеров.**  
   Выбираются кластеры с наименьшей средней оценкой (grade < 4).

3. **Генерация категорий и описание агентов.**  
   С помощью LLM (GigaChat) формируются:
   - Названия категорий, отражающие суть кластеров (например, "Проблемы с акциями и бонусными программами");
   - Описание ролей агентов, которые специализируются на анализе конкретных проблем.

4. **Создание подотчётов.**  
   Для каждого кластера генерируется подотчёт в Markdown.

5. **Формирование итогового отчёта.**  
   Все подотчёты объединяются в PDF-документ с помощью [md2pdf](https://github.com/jmaupetit/md2pdf).

-----
## Подготовка к запуску

Перед выполнением ноутбука необходимо установить следующие библиотеки:

In [74]:
%%capture --no-stderr
%pip install "arize-phoenix[embeddings]" langchain_gigachat pandas numpy md2pdf tqdm

In [ ]:
import getpass
import os

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Credentials от GigaChat")
    
scope = "GIGACHAT_API_PERS" # Или GIGACHAT_API_CORP / GIGACHAT_API_B2B

In [3]:
from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(
    model="GigaChat-Max",
    scope=scope,
    profanity_check=False,
    timeout=1000000
)

Подгружаем датасет с кластеризованными отзывами, который мы получили в [части 1](part_1.ipynb) 

In [2]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "mikhaildvoshansky/bank-reviews",
  "bank_reviews.csv",
).rename({"cluster_id": "__phoenix_cluster_id__"}, axis=1)

# Раскоментируйте строку ниже, чтобы загрузить свои данные с кластерами
# df = pd.read_pickle('bank_clusterized.pkl')
df

,Unnamed: 0,text,summary,title,grade,comment_count,is_countable,timestamp,__phoenix_cluster_id__
0,0,<p>Добрый день. Хоум банк передал все права в ...,"Пользователь сообщает, что его кредит был пере...",Передали мою кредитную карту в другой банк,1.0,0,NaN,2025-02-06 12:20:50.269000+00:00,21
1,1,<p>27.11 на кредитном счете 6889 были заблокир...,Пользователь сообщил о блокировке 10 000 рубле...,Незаконное удержание средств !!!!!!!,1.0,0,NaN,2025-02-06 12:20:50.269000+00:00,41
2,2,<p>Оформляла кредитную карту Хоум банк в июне ...,Пользователь оформил кредитную карту Хоум банк...,Не дали сертификат озон,3.0,0,NaN,2025-02-06 12:20:50.269000+00:00,0
3,3,"<p>Я пришел в офис, чтобы его закрыть досрочно...",Пользователь столкнулся с проблемой отсутствия...,Класс),5.0,0,0.0,2025-02-06 12:20:50.269000+00:00,11
4,4,<p>У меня есть кредит в Хоум банке.</p>\r\n<p>...,Пользователь столкнулся с проблемой при оплате...,Просрочка по вине банка,1.0,0,NaN,2025-02-06 12:20:50.269000+00:00,46
...,...,...,...,...,...,...,...,...,...
1015,1015,"<p>Захотел подать заявку на кредитную карту ""1...",**Краткий пересказ сообщения:**\n\nПользовател...,"Что надо знать перед оформлением кредитки ""120...",5.0,2,1.0,2025-02-06 12:20:50.269000+00:00,23
1016,1016,<p>Брала рассрочку в Хоум банке. Каждый месяц ...,Пользователь сообщает о проблемах с оплатой по...,Хоум Банк не дает совершить последний платеж,1.0,0,1.0,2025-02-06 12:20:50.269000+00:00,46
1017,1017,"<p>Здравствуйте, хочу поблагодарить банк Хоум ...",**Краткий пересказ:**\nПользователь благодарит...,Увеличили лимит на денежный перевод,5.0,0,0.0,2025-02-06 12:20:50.269000+00:00,8
1018,1018,<p>Здравствуйте! Я очень довольна что в этом б...,Пользователь выражает удовлетворение качеством...,Помогут даже в выходные дни,5.0,0,1.0,2025-02-06 12:20:50.269000+00:00,7


-----
### Выбираем кластера

In [4]:
with_grades = df.groupby(['__phoenix_cluster_id__'])['__phoenix_cluster_id__'].count().reset_index(name='count').join(
    df.groupby(['__phoenix_cluster_id__']).grade.mean(), on='__phoenix_cluster_id__'
).sort_values('count', ascending=False)
clusters = list(with_grades[with_grades['grade'] < 4]['__phoenix_cluster_id__'][:5])

In [5]:
clusters

[0, 21, 2, 14, 42]

------
### Создаем цепочку для названия кластеров

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from pydantic import BaseModel, Field

def format_messages(params):
    return {"messages": "\n--------\n".join(params['messages'])}
    
class CategoryInfo(BaseModel):
    """Информация о категории"""
    reasoning: str = Field(description="Рассуждения почему ты выбрал именно эту категорию")
    name: str = Field(description="Название категории")
    
category_parser = PydanticOutputParser(pydantic_object=CategoryInfo)

category_prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты помощник в составлении отчетов.
Твоя задача из списка сообщений ниже выявить одну общую категорию, к которой они относятся.

Ниже я приведу список сообщений
--------
{messages}
--------

Отвечай в в формате JSON
{format_instructions}
""")
]).partial(format_instructions=category_parser.get_format_instructions())

category_chain = (
        {"messages": RunnablePassthrough()} |
        RunnableLambda(format_messages) | 
        category_prompt | 
        llm.bind(temperature=0.1) | 
        category_parser
)

Проверяем работу цепочки

In [7]:
category_chain.invoke(
    list(df[df['__phoenix_cluster_id__'] == clusters[0]]['summary'])[:50]
)

CategoryInfo(reasoning='Все сообщения касаются проблем, связанных с невыполнением условий акций, неправильным начислением бонусов или кэшбэка, а также сложностями в коммуникации с поддержкой банка. Это объединяет их в одну общую категорию.', name='Проблемы с акциями и бонусами')

Теперь создаем названия категорий для всех 5 кластеров

In [110]:
from tqdm import tqdm

categories = {}

for i in tqdm(clusters):
    categories[i] = {
        "info": category_chain.invoke(
            list(df[df['__phoenix_cluster_id__'] == i]['summary'])[:50]
        )
    }

100%|██████████| 5/5 [00:13<00:00,  2.71s/it]


Посмотрим названия получившихся категорий

In [111]:
[category['info'].name for category in categories.values()]

['Проблемы с акциями и бонусными программами',
 'Слияние Хоум Банка и Совкомбанка',
 'Проблемы с получением банковских документов',
 'Банковские проблемы',
 'Проблемы с переводами и платежами']

--------
### Создаем цепочку для создания описания агента
**Мы не тестировали улучшает ли этот способ создание отчетов для категории**

Теория в том, что для разных тематик можно генерировать разный промпт, который будет
подставляться в промпт создания отчета

In [112]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field

class AgentInfo(BaseModel):
    """Информация об агенте"""

    name: str = Field(description="Название агента")
    agent_role_prompt: str = Field(description="Промпт агента")
    
agent_parser = PydanticOutputParser(pydantic_object=AgentInfo)

agent_generation = """Ты помощник в составлении отчетов.
Твоя задача создать описания агента исходя из заданной пользователем темы.
Определяется областью темы и конкретным именем агента, который может быть использован для создания отчета под конкретную тему. Агенты классифицируются по их области экспертизы.

Отвечай в формате JSON:
{format_instructions}

Примеры:

задача: "Инвестиции в акции Apple"
ответ: 
{{
    "name": "Финансовый агент",
    "agent_role_prompt": "Ты — ИИ-помощник аналитика финансов. Твоя основная задача — составлять всесторонние, продуманные, беспристрастные и методически структурированные финансовые отчёты на основе предоставленных данных и тенденций."
}}
задача: "перепродажа кроссовок"
ответ: 
{{
    "name":  "Агент бизнес-аналитики",
    "agent_role_prompt": "Ты — ИИ-помощник бизнес-аналитика. Твоя основная цель — создавать всесторонние, проницательные, беспристрастные и систематически структурированные бизнес-отчёты на основе предоставленных бизнес-данных, рыночных тенденций и стратегического анализа."
}}
задача: "интересные места в Тель-Авиве"
ответ:
{{
    "name":  "Туристический агент",
    "agent_role_prompt": "Ты — ИИ-помощник гида. Твоя основная задача — составлять увлекательные, информативные, беспристрастные и хорошо структурированные туристические отчёты по заданным местам, включая историю, достопримечательности и культурные особенности."
}}"""

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", agent_generation),
    ("user", "задача: \"{message}\"")
]).partial(format_instructions=agent_parser.get_format_instructions())

agent_chain = {"message": RunnablePassthrough()} | agent_prompt | llm.bind(temperature=0.2) | agent_parser

Тестируем цепочку

In [113]:
cluster = clusters[0]
agent_chain.invoke(categories[cluster]['info'].name)

AgentInfo(name='Финансовый агент', agent_role_prompt='Ты — ИИ-помощник финансового аналитика. Твоя основная задача — анализировать проблемы, связанные с акциями и бонусными программами, предоставлять рекомендации по улучшению ситуации и составлять подробные отчеты о финансовых рисках и возможностях.')

Заполняем описания агентов

In [115]:
for i in tqdm(clusters):
    categories[i]["agent"] = agent_chain.invoke(categories[i]['info'].name)

100%|██████████| 5/5 [00:09<00:00,  1.86s/it]


Смотрим результат

In [116]:
[i['agent'] for i in categories.values()]

[AgentInfo(name='Финансовый агент', agent_role_prompt='Ты — ИИ-помощник финансового аналитика. Твоя основная задача — анализировать проблемы, связанные с акциями и бонусными программами, предоставлять рекомендации по улучшению ситуации и составлять подробные отчеты о финансовых рисках и возможностях.'),
 AgentInfo(name='Корпоративный финансовый агент', agent_role_prompt='Ты — ИИ-помощник финансового аналитика. Твоя основная задача — составлять всеобъемлющие, объективные и структурированные отчеты о корпоративных слияниях и поглощениях, анализируя финансовую информацию, стратегии компаний и потенциальные риски.'),
 AgentInfo(name='Банковский агент', agent_role_prompt='Ты — ИИ-помощник банковского специалиста. Твоя основная задача — анализировать проблемы клиентов, связанные с банковскими документами, предоставлять информацию о возможных причинах задержек или ошибок, а также предлагать решения для устранения этих проблем.'),
 AgentInfo(name='Финансовый аналитик', agent_role_prompt='Ты — 

--------
### Создаем подотчеты

In [128]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

SUB_REPORT_PROBLEM = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """{agent_description}

**Цель:** Подготовить структурированный и детализированный подотчет на тему {theme}

### Формат отчета и требования:
- **Формат:** Markdown
- **Заголовки:** Используй заголовки с уровня 2 (##).
- **Анализ:** Выдели основные проблемы и предложения, которые пишут в своих отзывах пользователи.
- **Стиль:** Стремись к аналитическому и связному изложению;

Сообщения представлены ниже
Всего сообщений: {problem_count}
-------
{context}
-------

Твой отчет:
## {theme}
""")
    ]
)
report_chain = SUB_REPORT_PROBLEM | llm.bind(max_tokens=32000, temperature=0.2) | StrOutputParser()

In [118]:
categories['0']

{'info': CategoryInfo(reasoning='Все сообщения касаются различных проблем, связанных с участием в акциях банков, невыплатой обещанных бонусов, сложностью взаимодействия с поддержкой, а также недовольством условиями программ лояльности.', name='Проблемы с акциями и бонусными программами'),
 'agent': AgentInfo(name='Финансовый агент', agent_role_prompt='Ты — ИИ-помощник финансового аналитика. Твоя основная задача — анализировать проблемы, связанные с акциями и бонусными программами, предоставлять рекомендации по улучшению ситуации и составлять подробные отчеты о финансовых рисках и возможностях.')}

Тестируем цепочку

In [132]:
cluster = clusters[0]
problems = list(df[df['__phoenix_cluster_id__'] == cluster]['summary'])[:75]
print(report_chain.invoke({
    'theme': categories[cluster]['info'].name,
    'agent_description': categories[cluster]['agent'].agent_role_prompt,
    'context': "\n-------\n".join(problems),
    "problem_count": len(problems),
}))

### Анализ отзывов и выявленных проблем

#### Основные проблемы:

1. **Невыплата обещанных бонусов и сертификатов**:
   - Многие клиенты столкнулись с тем, что после выполнения всех условий акции (например, оформление карты, совершение покупок на определенную сумму) они не получили обещанные бонусы или сертификаты (особенно часто упоминается сертификат на Озон на 2500 рублей).
   - Часто банк отказывается признать свою ответственность, ссылаясь на партнеров (например, Озон), что создает путаницу и недовольство среди клиентов.

2. **Технические сбои и проблемы с начислением кэшбека**:
   - Некоторые пользователи сообщают о сложностях с оплатой картами в магазинах, что приводит к отмене транзакций и отсутствию начисления кэшбека.
   - Есть случаи, когда банк неправильно классифицирует категории покупок, что ведет к снижению размера кэшбека или его полному отсутствию.

3. **Недостаточная информативность и непрозрачность условий**:
   - Клиенты отмечают, что условия акций и бонусных програ

Запускаем генерацию подотчетов для всех категорий

In [133]:
for i in tqdm(clusters):
    problems = list(df[df['__phoenix_cluster_id__'] == i]['summary'])[:75]
    categories[i]['report'] = report_chain.invoke({
        'theme': categories[i]['info'].name,
        'agent_description': categories[i]['agent'].agent_role_prompt,
        'context': "\n-------\n".join(problems),
        "problem_count": len(problems),
    })

100%|██████████| 5/5 [01:22<00:00, 16.54s/it]


--------
### Склеиваем все подотчеты в PDF

In [134]:
from md2pdf.core import md2pdf
reports = ""

for i in clusters:
    reports += f"""
    
## {categories[i]['info'].name}

{categories[i]['report']}

"""
md2pdf(
    "report.pdf",
    md_content=f"""# Отчет по сообщениям

{reports}
""",
    css_file_path="styles.css",  # Updated path
    base_url=".",
)

---

## Заключение

Существует несколько направлений для дальнейшего улучшения pipeline'а:

- **Обработка большого объёма отзывов.**  
  На данный момент для генерации подотчёта используются только первые 75 отзывов. Это ограничение введено из-за большого размера первого кластера (150 отзывов). Для более полного анализа можно рассмотреть следующие подходы:
  1. **Итеративное расширение отчёта.** Начинать с генерации подотчёта по первому набору отзывов, а затем постепенно дополнять его, добавляя по 30–50 новых отзывов и прося LLM расширить уже созданный текст в режиме диалога (user-ai-user-ai-*).
  2. **Параллельная обработка чанков.** Разбить 150 отзывов на несколько частей (например, на 3 чанка), сформировать по отдельному подотчёту для каждой части и затем объединить их в единый подотчет.

- **Выделение ключевых тем.**  
  Перед генерацией подотчёта можно попросить LLM выделить темы, которые стоит раскрыть, исходя из суммаризаций. Дополнительно можно внедрить механизм *Human-in-the-Loop*, при котором специалист утвердит предложенные темы или скорректирует их, что позволит добиться более точного отражения проблематики.

- **Многоклассовая категоризация.**  
  Текущий pipeline присваивает каждому отзыву лишь одну категорию проблем, что может ограничивать полноту анализа. В данный момент мы разрабатываем подход, позволяющий присваивать отзывам несколько тематических категорий. Эти категории могут быть заранее определены или выявляться динамически и добавляться в дерево категорий.

Мы продолжаем работать над совершенствованием данного pipeline. Если удастся успешно реализовать предложенные улучшения, результаты будут опубликованы в ближайшее время.  
**To be continued…**
